# Web scraping financial market data from Yahoo Finance

In [1]:
from collections import defaultdict
import bs4
import pandas as pd
import requests

pd.set_option('max_row', 1000)
pd.set_option('max_columns', 1000)

def parse_url_to_df(url, ticker):
    """Shared utility function to convert data from URL to a dataframe.

    Used on Yahoo Finance Key Statistics and Quote pages.
    """
    res = requests.get(url)
    res.raise_for_status()
    soup = bs4.BeautifulSoup(res.text, 'html.parser')
    stats = defaultdict(dict)

    # gather field-value pairs into a `stats` dict
    for x in soup.findAll('tr'):
        stuff = x.findAll('td')
        field, val = stuff[0].text, stuff[1].text
        stats[field] = val
        #for y in x.findAll('td'):
            #print(y.text)

    df = pd.DataFrame.from_dict(stats, orient='index', columns=[ticker])
    return df
    
def get_key_stats(ticker):
    """Returns key statistics for a single stock."""
    url = rf'https://finance.yahoo.com/quote/{ticker}/key-statistics'
    df = parse_url_to_df(url, ticker)    
    return df

def get_quote_stats(ticker):    
    """Returns key statistics for a single stock."""
    url = rf'https://finance.yahoo.com/quote/{ticker}'
    df = parse_url_to_df(url, ticker)
    return df


def get_group(tickers, func, axis=1):
    """Consolidates stats for a list of tickers by dispatching a function that returns a dataframe.
    
    Example
    -------
    >>> stats = get_group(['EVH', 'TSLA', 'UAL', 'DAL', 'AAL', 'BOX', 'DBX'], get_key_stats)
    >>> all_execs = get_group(['EVH', 'TSLA', 'UAL', 'DAL', 'AAL', 'BOX', 'DBX'], get_execs, axis=0)

    ... where get_key_stats and get_execs are functions. 
    """

    data = pd.DataFrame()

    for stock in tickers:
        data = pd.concat([data, func(stock)], axis=axis)
        
    return data.T

def get_execs(ticker):
    """Returns dataframe of exec for single company given ticker.
    
    Example
    -------
    >>> tsla_execs = get_execs('TSLA')
    >>> tsla_execs
    |    | name                    | title                                       | exercised   |   year_born | co   |
    |---:|:------------------------|:--------------------------------------------|:------------|------------:|:-----|
    |  0 | Mr. Elon R. Musk        | Co-Founder, CEO & Director                  | N/A         |        1972 | TSLA |
    |  1 | Mr. Jeffrey B. Straubel | Sr. Advisor                                 | 9.41M       |        1976 | TSLA |
    |  2 | Mr. Jerome  Guillen     | Pres of Automotive Division                 | 1.63M       |        1973 | TSLA |
    |  3 | Mr. Zachary  Kirkhorn   | Chief Financial Officer                     | N/A         |        1985 | TSLA |
    |  4 | Mr. Vaibhav  Taneja     | Corp. Controller & Chief Accounting Officer | N/A         |        1978 | TSLA |

    """
    
    URL = rf'https://finance.yahoo.com/quote/{ticker}/profile'
    res = requests.get(URL)
    res.raise_for_status()
    soup = bs4.BeautifulSoup(res.text, 'html.parser')
    stats = defaultdict(dict)
    
    for x in soup.findAll('tr')[1:]:
        stuff = x.findAll('td')
        name, title, pay, exercised, year_born = stuff[0].text, stuff[1].text, stuff[2].text, stuff[3].text, stuff[4].text
        stats[name] = {'name': name, 'title': title, 'exercised': exercised, 'year_born': year_born}

    df = pd.DataFrame.from_dict(stats, orient='index').reset_index(drop=True)
    df['co'] = ticker
    return df

def get_historical_quotes(ticker):
    url = rf'https://finance.yahoo.com/quote/{ticker}/history'
    res = requests.get(url)
    res.raise_for_status()
    soup = bs4.BeautifulSoup(res.text, 'html.parser')
    stats = defaultdict(dict)

    # gather field-value pairs into a `stats` dict
    for x in soup.findAll('tr')[1:-1]:
        stuff = x.findAll('td')
        date, x_open, hi, lo = stuff[0].text, stuff[1].text, stuff[2].text, stuff[3].text
        close, adj_close, volume = stuff[4].text, stuff[5].text, stuff[6].text
        stats[date] = {'open': x_open, 'high': hi, 'low': lo, 'close': close, 'adj_close': adj_close, 'volume': volume}
    #     for y in x.findAll('td'):
    #         print(y.text)

    df = pd.DataFrame.from_dict(stats, orient='index')
    df.index.name = 'date'
    df['co'] = ticker
    return df

In [2]:
stocks = ['EVH', 'TSLA', 'UAL', 'DAL', 'AAL', 'BOX', 'DBX']

key_stats = get_group(stocks, get_key_stats)
quote_stats = get_group(stocks, get_quote_stats)
all_execs = get_group(stocks, get_execs, axis=0).T
hist = get_historical_quotes('TSLA')

In [3]:
# key_stats
# quote_stats
# all_execs

In [4]:
key_stats

,Market Cap (intraday) 5,Enterprise Value 3,Trailing P/E,Forward P/E 1,PEG Ratio (5 yr expected) 1,Price/Sales (ttm),Price/Book (mrq),Enterprise Value/Revenue 3,Enterprise Value/EBITDA 6,Beta (5Y Monthly),52-Week Change 3,S&P500 52-Week Change 3,52 Week High 3,52 Week Low 3,50-Day Moving Average 3,200-Day Moving Average 3,Avg Vol (3 month) 3,Avg Vol (10 day) 3,Shares Outstanding 5,Float,% Held by Insiders 1,% Held by Institutions 1,"Shares Short (Feb 13, 2020) 4","Short Ratio (Feb 13, 2020) 4","Short % of Float (Feb 13, 2020) 4","Short % of Shares Outstanding (Feb 13, 2020) 4","Shares Short (prior month Jan 14, 2020) 4",Forward Annual Dividend Rate 4,Forward Annual Dividend Yield 4,Trailing Annual Dividend Rate 3,Trailing Annual Dividend Yield 3,5 Year Average Dividend Yield 4,Payout Ratio 4,Dividend Date 3,Ex-Dividend Date 4,Last Split Factor 2,Last Split Date 3,Fiscal Year Ends,Most Recent Quarter (mrq),Profit Margin,Operating Margin (ttm),Return on Assets (ttm),Return on Equity (ttm),Revenue (ttm),Revenue Per Share (ttm),Quarterly Revenue Growth (yoy),Gross Profit (ttm),EBITDA,Net Income Avi to Common (ttm),Diluted EPS (ttm),Quarterly Earnings Growth (yoy),Total Cash (mrq),Total Cash Per Share (mrq),Total Debt (mrq),Total Debt/Equity (mrq),Current Ratio (mrq),Book Value Per Share (mrq),Operating Cash Flow (ttm),Levered Free Cash Flow (ttm),Shares Short 4,Short Ratio 4,Short % of Float 4,Short % of Shares Outstanding 4,Shares Short (prior month ) 4
EVH,591.59M,945.36M,N/A,-744.00,-0.80,0.70,0.68,1.12,-15.55,1.07,-36.48%,6.79%,14.79,5.50,10.33,8.69,1.42M,1.07M,84.72M,77.23M,1.20%,103.26%,14.47M,13.92,19.38%,17.08%,15.58M,N/A,N/A,N/A,N/A,N/A,0.00%,N/A,N/A,N/A,N/A,"Dec 30, 2019","Dec 30, 2019",-35.68%,-12.17%,-4.00%,-28.85%,846.38M,10.28,22.50%,197.55M,-60.8M,-301.97M,-3.67,N/A,102.81M,1.21,368.79M,39.70,1.19,10.90,-42.65M,25.45M,NaN,NaN,NaN,NaN,NaN
TSLA,108.85B,137.49B,N/A,43.74,2.40,4.43,16.63,5.59,62.87,0.48,141.81%,6.79%,968.99,176.99,725.80,422.06,18.86M,16.9M,184.39M,143.77M,20.66%,54.06%,18.39M,0.76,N/A,10.14%,24.95M,N/A,N/A,N/A,N/A,N/A,0.00%,N/A,N/A,N/A,N/A,"Dec 30, 2019","Dec 30, 2019",-3.51%,0.32%,0.16%,-10.75%,24.58B,138.86,2.20%,4.07B,2.19B,-862M,-4.92,-25.00%,6.27B,34.56,14.7B,181.22,1.13,36.56,2.4B,1.45B,NaN,NaN,NaN,NaN,NaN
UAL,11.6B,28.42B,4.04,3.33,0.35,0.27,1.02,0.66,4.23,1.37,-36.76%,6.79%,96.03,45.92,73.61,85.04,4.98M,12.61M,247.95M,232.74M,0.42%,101.09%,11.88M,2.34,6.31%,4.73%,12.04M,N/A,N/A,N/A,N/A,N/A,0.00%,"Jan 22, 2008","Jan 06, 2008",4:1,"May 20, 1996","Dec 30, 2019","Dec 30, 2019",6.96%,10.56%,5.62%,27.90%,43.26B,167.15,3.80%,14.78B,6.72B,3.01B,11.58,39.00%,4.94B,19.94,20.45B,177.35,0.55,45.90,6.91B,1.24B,NaN,NaN,NaN,NaN,NaN
DAL,28.4B,44.36B,5.96,5.52,0.73,0.60,1.81,0.94,4.98,1.29,-10.23%,6.79%,63.44,42.42,54.84,56.48,9.02M,23.55M,646.74M,635.01M,50.04%,0.00%,22.51M,3.05,3.87%,3.52%,20.89M,1.61,3.58%,1.50,3.28%,1.80,20.62%,"Mar 11, 2020","Feb 18, 2020",N/A,N/A,"Dec 30, 2019","Dec 30, 2019",10.14%,14.15%,6.66%,32.83%,47.01B,72.21,6.50%,12.92B,8.91B,4.77B,7.30,7.80%,2.89B,4.54,17.99B,117.17,0.41,23.99,8.42B,2.27B,NaN,NaN,NaN,NaN,NaN
AAL,6.28B,36.42B,3.89,2.69,0.28,0.14,N/A,0.80,5.87,1.90,-50.17%,6.79%,35.24,14.46,25.32,27.45,13.91M,43.34M,428.2M,419.78M,40.01%,5.17%,31.12M,2.83,8.37%,7.30%,42.66M,0.4,2.49%,0.40,2.50%,1.05,10.55%,"Feb 18, 2020","Feb 03, 2020",0:1,"Dec 08, 2013","Dec 30, 2019","Dec 30, 2019",3.68%,8.50%,4.03%,N/A,45.77B,103.23,3.40%,12.56B,6.21B,1.69B,3.79,27.80%,3.83B,8.98,33.44B,N/A,0.45,-0.28,3.82B,-17.75M,NaN,NaN,NaN,NaN,NaN
BOX,1.96B,2.44B,N/A,20.70,4.69,2.82,88.11,3.50,-30.48,1.41,-25.90%,6.79%,21.19,12.46,15.61,16.41,1.72M,2.54M,149.63M,146.18M,3.16%,79.16%,7.09M,4.86,5.00%,4.71%,6.33M,N/A,N/A,N/A,N/A,N/A,0.00%,N/A,N/A,N/A,N/A,"Jan 30, 2020","Jan 30, 2020",-20.73%,-20.03%,-10.86%,-536.99%,696.26M,4.71,12.10%,480.69M,-80.05M,-144.35M,-0.98,N/A,195.59M,1.3,424.54M,"1,898.92",0.79,0.15,44.71M,126.84M,NaN,NaN,NaN,NaN,NaN
DBX,7.4B,7.98B,N/A,19.63,0.78,4.46,9.11,4.80,84.87,N/A,-14.27%,6.7

In [5]:
quote_stats

,Previous Close,Open,Bid,Ask,Day's Range,52 Week Range,Volume,Avg. Volume,Market Cap,Beta (5Y Monthly),PE Ratio (TTM),EPS (TTM),Earnings Date,Forward Dividend & Yield,Ex-Dividend Date,1y Target Est
EVH,7.94,7.63,0.00 x 800,7.46 x 800,7.12 - 7.88,5.50 - 14.79,"999,394","1,416,349",591.592M,1.07,N/A,-3.67,"Feb 24, 2020",N/A (N/A),N/A,15.44
TSLA,703.48,605.39,0.00 x 3100,0.00 x 1800,605.00 - 663.00,176.99 - 968.99,"17,073,740","18,857,003",108.85B,0.48,N/A,-4.92,"Apr 21, 2020 - Apr 26, 2020",N/A (N/A),N/A,504.33
UAL,52.10,49.58,0.00 x 1200,0.00 x 800,45.92 - 50.99,45.92 - 96.03,"12,307,563","4,978,306",11.714B,1.37,4.04,11.58,"Apr 13, 2020 - Apr 19, 2020",N/A (N/A),"Jan 06, 2008",97.13
DAL,45.89,43.04,0.00 x 900,44.11 x 900,42.42 - 45.01,42.42 - 63.44,"25,599,851","9,023,767",28.402B,1.29,5.96,7.30,"Apr 07, 2020 - Apr 12, 2020",1.61 (3.58%),"Feb 18, 2020",66.63
AAL,15.97,14.87,0.00 x 900,0.00 x 2200,14.46 - 15.79,14.46 - 35.24,"42,558,021","13,911,763",6.284B,1.90,3.89,3.79,"Apr 23, 2020 - Apr 27, 2020",0.40 (2.49%),"Feb 03, 2020",29.93
BOX,14.68,13.99,0.00 x 900,0.00 x 900,13.00 - 14.11,12.46 - 21.19,"3,021,583","1,715,808",1.964B,1.41,N/A,-0.98,"May 31, 2020 - Jun 04, 2020",N/A (N/A),N/A,18.33
DBX,19.41,18.46,0.00 x 1400,0.00 x 1800,17.63 - 18.46,16.08 - 26.20,"5,276,806","5,532,198",7.401B,N/A,N/A,-0.13,"Feb 19, 2020",N/A (N/A),N/A,28.92


In [6]:
all_execs

,name,title,exercised,year_born,co
0,Mr. Frank J. Williams,"Co-Founder, CEO & Chairman",N/A,1967,EVH
1,Mr. Seth Blackley,"Co-Founder, Pres & Director",2.83M,1979,EVH
2,Mr. Thomas Peterson III,Co-Founder & COO,1.66M,1970,EVH
3,Mr. Jonathan D. Weinberg,"Chief Legal Officer, Gen. Counsel & Sec.",591.84k,1968,EVH
4,Dr. Norman C. Payson,Advisor,N/A,1949,EVH
0,Mr. Elon R. Musk,"Co-Founder, CEO & Director",N/A,1972,TSLA
1,Mr. Jeffrey B. Straubel,Sr. Advisor,9.41M,1976,TSLA
2,Mr. Jerome Guillen,Pres of Automotive Division,1.63M,1973,TSLA
3,Mr. Zachary Kirkhorn,Chief Financial Officer,N/A,1985,TSLA
4,Mr. Vaibhav Taneja,Corp. Controller & Chief Accounting Officer,N/A,1978,TSLA


In [7]:
hist

,open,high,low,close,adj_close,volume,co
date,,,,,,,
"Mar 09, 2020",605.39,663.00,605.00,608.00,608.00,"16,879,800",TSLA
"Mar 06, 2020",690.00,707.00,684.27,703.48,703.48,"12,662,900",TSLA
"Mar 05, 2020",723.77,745.75,718.07,724.54,724.54,"10,852,700",TSLA
"Mar 04, 2020",763.96,766.52,724.73,749.50,749.50,"15,049,000",TSLA
"Mar 03, 2020",805.00,806.98,716.11,745.51,745.51,"25,784,000",TSLA
"Mar 02, 2020",711.26,743.69,686.67,743.62,743.62,"20,195,000",TSLA
"Feb 28, 2020",629.70,690.52,611.52,667.99,667.99,"24,564,200",TSLA
"Feb 27, 2020",730.00,739.77,669.00,679.00,679.00,"24,277,200",TSLA
"Feb 26, 2020",782.50,813.31,776.11,778.80,778.80,"14,085,500",TSLA


# Scratch

In [8]:
from datetime import datetime

def from_e_to_dt(epoch):
    """Convert from epoch time to datetime."""
    dt = datetime.fromtimestamp(epoch)
    return dt

def from_dt_to_e(dt):
    """Convert from datetime to seconds since epoch.
    >>> from_e_to_dt(1583798400)
    >>> from_dt_to_e(from_e_to_dt(1583798400))
    """
    epoch = int(dt.timestamp())
    return epoch


In [9]:
# download doesn't work
# Yahoo Finance blocks requests that aren't coming from authorized browser

import wget

ticker = 'TSLA'
period1 = 1583366400
period2 = 1583798400
url_template = rf'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval=1d&events=history&crumb=WHQcTw1fgKa'
print(url_template)
# filename = wget.download(url_template)

https://query1.finance.yahoo.com/v7/finance/download/TSLA?period1=1583366400&period2=1583798400&interval=1d&events=history&crumb=WHQcTw1fgKa
